# **주제 분석(Topic Analysis)**

주제 분석은 **텍스트 데이터**에서 **반복적으로 나타나는 단어 그룹이나 주제**를 추출하고 분석하는 자연어 처리(NLP)의 기술입니다. 다양한 비지도 학습 및 지도 학습 방법을 활용해 데이터를 요약하고 군집화합니다.

---

## **1. 주제 분석의 개념과 목적**  

### **1.1 주제(Topic)란?**  
- **텍스트 데이터에서 반복적으로 나타나는 단어 그룹** 또는 **주제**를 의미합니다.  
- **예시**:  
   - 뉴스 기사 → "경제", "스포츠", "정치"  
   - 고객 리뷰 → "가격", "서비스 품질", "상품 기능"

### **1.2 목적**  
- **숨겨진 구조 이해**: 텍스트 데이터에서 숨겨진 주제를 찾아내기.  
- **문서 요약**: 방대한 텍스트를 요약해 주요 내용 파악.  
- **유사 텍스트 군집화**: 비슷한 내용의 문서 그룹화.  
- **정보 검색 개선**: 추천 시스템 및 검색 엔진 성능 개선.  

---

## **2. 주제 분석 유형**

### **2.1 비지도 학습 기반 토픽 모델링**  
- **정의**: 토픽을 자동으로 추출하는 방식입니다.  
- **기술**:  
   - **LSA (Latent Semantic Analysis)**: SVD(특이값 분해)를 기반으로 의미 추출  
   - **LDA (Latent Dirichlet Allocation)**: 확률적 모델로 문서의 토픽 분포를 학습  

### **2.2 지도 학습 기반 토픽 모델링**  
- **정의**: 미리 정의된 주제나 레이블로 데이터를 분류하는 방식입니다.  
- **기술**:  
   - 머신러닝 및 딥러닝 모델 사용  

---

## **3. LSA (Latent Semantic Analysis)**  

### **3.1 개념**  
- **잠재 의미 분석**: 단어-문서 행렬을 SVD(특이값 분해)로 분해하여 의미를 추출하는 방법입니다.  
- **핵심 아이디어**: 단어의 빈도뿐 아니라 **의미 기반**으로 문서를 재배치합니다.  

---

### **3.2 Truncated SVD를 활용한 주제 분석**  

#### **단계별 과정**  

1. **데이터 벡터화 (Bag-of-Words)**  
   ```python
   from sklearn.feature_extraction.text import CountVectorizer
   vec = CountVectorizer(max_features=1000)
   x_arr = vec.fit_transform([ " ".join(tokens) for tokens in tokens_list ]).A
   x_arr.shape
   ```

2. **SVD 적용**  
   ```python
   from sklearn.decomposition import TruncatedSVD
   svd = TruncatedSVD(n_components=20, random_state=SEED)
   u_arr = svd.fit_transform(x_arr)
   ```

3. **결과 확인**  
   - **U 행렬**: 문서-주제 관계  
   - **Σ (특이값)**: 주제의 중요도  
   - **Vt 행렬**: 주제-단어 관계  

4. **특이값 시각화 (엘보우 방법)**  
   ```python
   fig , ax = plt.subplots()
   ax.plot( range(1,21), svd.singular_values_, marker="o")
   plt.show()
   ```

---

## **4. LDA (Latent Dirichlet Allocation)**  

### **4.1 개념**  
- **잠재 디리클레 할당**: 문서가 여러 주제의 혼합으로 구성되며, 각 주제는 특정 단어 분포를 따른다고 가정하는 확률 모델입니다.  

### **4.2 LDA 수행 과정**  
1. **토픽 개수 \( k \) 설정**  
2. **모든 단어를 주제에 랜덤하게 할당**  
3. **문서와 주제의 확률 분포 계산**  
4. **단어를 가장 높은 확률의 주제에 재할당**  
5. **반복**  

---

### **4.3 Gensim으로 LDA 모델 학습**  

#### **1. Dictionary 생성**  
```python
from gensim.corpora import Dictionary
dic = Dictionary(tokens_list)
dic.filter_extremes(no_below=10)
```

#### **2. Corpus 생성**  
```python
corpus = [ dic.doc2bow(tokens) for tokens in tokens_list ]
```

#### **3. LDA 모델 학습**  
```python
from gensim.models import LdaModel
model = LdaModel(corpus=corpus, num_topics=8, id2word=dic, passes=10, random_state=SEED)
```

#### **4. 토픽별 상위 단어 확인**  
```python
model.print_topics(num_topics=8)
```

---

## **5. 주제 다양도와 주제 응집도**  

### **5.1 주제 다양도 (Topic Diversity)**  
- **정의**: 토픽 간 단어 중복 정도를 평가  
- **계산 방법**:  
   $
   \text{다양도} = \frac{\text{합집합의 단어 수}}{\text{주제 수} \times \text{상위 단어 수}}
   $

```python
topn = 10
s = set()
for k in range(8):
    arr = np.array(model.show_topic(k, topn))
    s.update(arr[:,0])
    
score = len(s) / (8 * topn)
```

---

### **5.2 주제 응집도 (Topic Coherence)**  
- **정의**: 각 토픽을 구성하는 단어들의 **연관성 정도**를 평가합니다.  
- **0~1 사이의 값**이며, 1에 가까울수록 응집도가 높습니다.  

```python
from gensim.models import CoherenceModel
coh = CoherenceModel(model=model, texts=tokens_list, dictionary=dic)
coh.get_coherence()
```

---

## **6. 문서별 주제 확률 분포**  

### **벡터화된 주제 확률 생성**  
```python
df_lda = []
for lst in model.get_document_topics(corpus):
    lst = np.array(lst)
    tmp = np.zeros(8)
    idx = lst[:,0].astype(int)
    tmp[idx] = lst[:,1]
    df_lda.append(tmp)

df_lda = pd.DataFrame(df_lda)
```

### **문서별 주요 주제 확인**  
```python
df_lda.idxmax(axis=1)
```

---

## **7. LDA 시각화**

### **pyLDAvis 활용**  
LDA 모델의 결과를 시각화하여 주제 간 관계와 단어 분포를 확인합니다.  

```python
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(model, corpus, dic)
pyLDAvis.display(vis)
```

- **왼쪽 그래프**: 토픽 간 관계와 중요도 시각화  
- **오른쪽 그래프**: 각 토픽의 주요 단어 분포 확인  

---

## **핵심 요약**  
1. **주제 분석**은 텍스트 데이터에서 숨겨진 주제를 추출하는 기법입니다.  
2. **LSA**와 **LDA**는 대표적인 토픽 모델링 기법입니다.  
3. **주제 다양도**와 **주제 응집도**를 평가해 모델의 품질을 확인합니다.  
4. **pyLDAvis**를 활용하면 LDA 결과를 직관적으로 시각화할 수 있습니다.  

# LSA VS LDA

**LSA**와 **LDA**는 둘 다 텍스트 데이터에서 **숨겨진 주제(Topic)**를 찾기 위한 **주제 모델링 기법**
하지만 **접근 방식**과 **결과 해석**에 차이가 있다.   

---

## **LSA (Latent Semantic Analysis): 의미 기반 주제 분석**  

### **핵심 개념**  
- **LSA**는 **수학적 방법(SVD, 특이값 분해)**을 사용해 텍스트에서 **단어의 의미**를 분석합니다.  
- **문서-단어 행렬**을 분해해서 **의미의 숨겨진 차원(주제)**을 찾아냅니다.  

---

### **비유로 이해하기**  
**LSA는 '성적표를 분석하는 방법'**과 비슷합니다.

- **학생(문서)**과 **과목(단어)**의 성적표(빈도수)를 **행렬**이라고 생각해봅시다.  
- 여기서 **성적표를 분석**하면, 학생들이 어떤 **숨겨진 능력(주제)**을 가지고 있는지를 찾아낼 수 있습니다.  

#### 예시:  
- 성적표에는 수학, 물리, 화학, 미술, 음악 점수가 있습니다.  
- **LSA**를 사용하면 다음과 같은 숨겨진 주제(능력)를 찾아냅니다:  
   - **주제 1: 이과형 능력** → 수학, 물리, 화학 점수에서 영향  
   - **주제 2: 예체능형 능력** → 미술, 음악 점수에서 영향  

이처럼 **LSA**는 "문서-단어 행렬"을 수학적으로 분해하여 숨겨진 주제를 찾아내지만, 단순히 **수학적 패턴**만을 보기 때문에 주제의 의미는 사용자가 해석해야 합니다.

---

### **장점**  
- 계산이 비교적 간단하고 빠르다.  
- 의미적으로 비슷한 단어를 **하나의 주제**로 묶어준다.  

### **단점**  
- 수학적 패턴만 사용하므로, **확률적 모델**에 비해 해석이 어렵다.  
- 데이터에 **노이즈**가 많으면 결과가 왜곡될 수 있다.  

---

## **LDA (Latent Dirichlet Allocation): 확률 기반 주제 분석**  

### **핵심 개념**  
- **LDA**는 **확률 모델**을 사용해 문서가 **여러 주제의 혼합**으로 이루어져 있다고 가정합니다.  
- 각 단어는 특정 주제에서 나올 **확률**에 따라 등장한다고 가정합니다.  

---

### **비유로 이해하기**  
**LDA는 '요리의 레시피'**를 분석하는 방법과 비슷합니다.

- 요리는 여러 **재료(단어)**가 섞여서 만들어집니다.  
- **레시피(문서)**를 보고, 그 안에 어떤 **요리(주제)**가 숨어 있는지 확률적으로 분석합니다.  

#### 예시:  
- **요리 1: 스파게티** → 재료: 면(70%), 토마토(20%), 치즈(10%)  
- **요리 2: 샐러드** → 재료: 채소(60%), 드레싱(30%), 치즈(10%)  
- **레시피 1**에서 **면, 토마토, 채소**가 등장한다면, LDA는 다음과 같이 확률적으로 추정합니다:  
   - **스파게티(70%)**, **샐러드(30%)**  

이처럼 **LDA**는 문서가 여러 주제로 이루어졌다고 가정하고, 주제와 단어의 **확률 분포**를 학습합니다.  

---

### **장점**  
- 확률 모델을 기반으로 하므로 결과가 더 해석 가능하고 신뢰도가 높다.  
- 문서가 **여러 주제의 혼합**으로 구성될 수 있음을 고려한다.  

### **단점**  
- 계산이 LSA보다 복잡하고 시간이 오래 걸린다.  
- 주제 개수(토픽 수)를 **사전에 설정**해야 한다.  

---

## **LSA vs LDA의 차이점 요약**

| **구분**               | **LSA**                                | **LDA**                                |
|------------------------|---------------------------------------|---------------------------------------|
| **접근 방식**          | **수학적 분해 (SVD)**                 | **확률 모델**                         |
| **숨겨진 주제**        | 수학적 패턴에 의해 결정됨             | 단어의 확률 분포를 기반으로 결정됨     |
| **해석 가능성**        | 해석이 어렵고 주제의 의미가 불분명함   | 주제와 단어 확률을 통해 해석이 명확함 |
| **문서와 주제의 관계** | 문서는 **하나의 주제**로 압축됨         | 문서는 **여러 주제의 혼합**으로 표현됨 |
| **계산 복잡도**        | 비교적 간단하고 빠름                  | 계산이 복잡하고 시간이 더 걸림         |

---

## **간단한 예시 비교**

| **문서**                     | **LSA 결과**                       | **LDA 결과**                           |
|-----------------------------|----------------------------------|--------------------------------------|
| "강아지, 고양이, 귀엽다"     | 주제 1: **반려동물**                | 주제 1: **반려동물(70%)**, 주제 2: **감정(30%)** |
| "가격, 품질, 배송이 좋다"     | 주제 2: **상품 리뷰**               | 주제 3: **품질(50%)**, 주제 4: **배송(50%)**   |

**해석**:  
- **LSA**는 문서를 **단일 주제**로 분류하는 느낌입니다.  
- **LDA**는 문서가 여러 주제의 **혼합**임을 고려하여 확률적으로 표현합니다.  

---

## **결론: 언제 LSA와 LDA를 사용할까?**  

1. **빠르게 주제를 분석하고 싶다면 → LSA**  
   - 계산이 간단하고 빠르므로 초기 분석에 적합합니다.  

2. **더 정확하고 해석 가능한 결과를 원한다면 → LDA**  
   - 확률적 모델을 사용하므로 주제의 의미가 명확하고 결과가 신뢰할 수 있습니다.  

---

이처럼 LSA는 **수학적 분해**를 통해 숨겨진 주제를 찾는 반면, LDA는 **확률적 모델**을 통해 더 직관적이고 해석 가능한 주제를 찾아낸다.